In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import csv
import os
from datetime import datetime

service = Service('C:/chromedriver.exe')  # 크롬 드라이버 경로
chrome_options = webdriver.ChromeOptions()
browser = webdriver.Chrome(service=service, options=chrome_options)

# 리뷰어 URL 리스트
reviewer_urls = [

    {
        "name": "다블리_복합성",
        "url": "https://www.oliveyoung.co.kr/store/mypage/getReviewerProfile.do?key=RzRieDB1L2ZpelprbUlTOHY4eC9mZz09&t_page=%EC%83%81%ED%92%88%EC%83%81%EC%84%B8&t_click=%EB%A6%AC%EB%B7%B0%EC%96%B4_%EB%A6%AC%EB%B7%B0%EC%96%B4%ED%94%84%EB%A1%9C%ED%95%84&t_profile_name=%EB%8B%A4%EB%B8%94%EB%A6%AC"
    },
    {
      "name": "올챙이포뇨_건성",
        "url": "https://www.oliveyoung.co.kr/store/mypage/getReviewerProfile.do?key=S1l5c0U5dzVGSEpwYVM3N3N0RHp0dz09&t_page=%EC%83%81%ED%92%88%EC%83%81%EC%84%B8&t_click=%EB%A6%AC%EB%B7%B0%EC%96%B4_%EB%A6%AC%EB%B7%B0%EC%96%B4%ED%94%84%EB%A1%9C%ED%95%84&t_profile_name=%EC%98%AC%EC%B1%99%EC%9D%B4%ED%8F%AC%EB%87%A8"
    }
]

##### 안전한 요소 추출 함수 정의

In [7]:
def safe_find(css_selector):
    try:
        return browser.find_element(By.CSS_SELECTOR, css_selector).text.strip()
    except:
        return None

##### 스크롤 함수 정의

In [8]:
def smart_scroll_to_load_reviews(max_scrolls, delay):
    last_count = 0
    stable_count = 0
    for i in range(max_scrolls):
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(delay)
        boxes = browser.find_elements(By.CSS_SELECTOR, '.rw-box')

        current_count = len(boxes)
        if current_count == last_count:
            stable_count += 1
            if stable_count >= 2:
                print(f"스크롤 반복 중단, (총 {i+1}회 스크롤)")
                break
        else:
            stable_count = 0
        last_count = current_count


##### 리뷰 파싱 함수 정의

In [9]:
def parse_review_box(box):
    # 박스 내부용 (리뷰)
    def safe_find_in_box(css_selector):
        try:
            return box.find_element(By.CSS_SELECTOR, css_selector).text.strip()
        except:
            return None

    # 기본 항목
    brand = safe_find_in_box(".rw-box-figcaption__brand")
    product_name = safe_find_in_box(".rw-box-figcaption__name")
    product_option = safe_find_in_box(".rw-box-figcaption__sub")
    rating = safe_find_in_box(".point")
    review_text = safe_find_in_box(".rw-box__description")

    help_count_raw = safe_find_in_box(".rw-box__help .num")
    try:
        help_count = int(help_count_raw.replace(',', '')) if help_count_raw else 0
    except:
        help_count = 0


    # 작성일
    try:
        date = box.find_element(By.CSS_SELECTOR, ".review_point_text span").text.strip()
    except:
        date = None

    # 체험단 여부 (Y/N 변환)
    try:
        from_experience_group = "Y" if box.find_element(By.CSS_SELECTOR, ".ico_oyGroup") else "N"
    except:
        from_experience_group = "N"

    # 매장 구매 여부 (Y/N 변환)
    try:
        is_offline_store = "Y" if box.find_element(By.CSS_SELECTOR, ".ico_offlineStore") else "N"
    except:
        is_offline_store = "N"

    # 재구매 & 한달이상사용 여부
    second_line_spans = box.find_elements(By.CSS_SELECTOR, ".rw-box__second-line span")
    second_line_texts = [s.text.strip() for s in second_line_spans]

    used_over_month = "Y" if "한달이상사용" in second_line_texts else "N"
    repurchase = "Y" if "재구매" in second_line_texts else "N"

    # 딕셔너리 저장
    review_data = {
        "brand_name": brand,
        "product_name": product_name,
        "product_option": product_option,
        "score": rating,
        "date": date,
        "purchase_store": is_offline_store,
        "used_over_month": used_over_month,
        "is_repurchase": repurchase,
        "is_experience": from_experience_group,
        "review": review_text,
        "help_count": help_count,
    }

    return review_data

##### csv 저장용 데이터 정리 함수 정의

In [10]:
def clean_for_csv(data_dict):
    # 딕셔너리 초기화
    cleaned = {}

    # 원본 딕셔너리를 key-value로 순회
    for k, v in data_dict.items():
        # 값이 비어있으면 공백문자열
        if v is None:
            cleaned[k] = ''
        #  리스트면 쉼표로 이어붙임
        elif isinstance(v, list):
            cleaned[k] = ', '.join(v)
        # 나머지는 문자열로 바꾸고 줄바꿈 제거
        else:
            cleaned[k] = str(v).replace('\r\n', ' ').replace('\n', ' ').strip()
    return cleaned

##### 전체 크롤링 실행 후 CSV로 저장

In [11]:
# 중간 저장 간격
save_interval = 200

# 열 순서 고정
field_order = [
    'reviewer_name', 'reviewer_introduce', 'reviewer_type', 'top_reviewer_field', 'brand_name', 'product_name', 'product_option',
    'score', 'date', 'purchase_store', 'used_over_month', 'is_repurchase', 'is_experience',
    'review', 'help_count'
]

# 전체 크롤링
for user in reviewer_urls:

    print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] 리뷰 크롤링 시작!")

    name = user["name"]
    url = user["url"]

    # 예외처리
    try:
        browser.get(url)
        browser.implicitly_wait(2)
    except Exception as e:
        print(f"[ERROR] {name} 페이지 열기 실패: {e}")
        continue

    # 프로필 정보 파싱
    reviewer_name = safe_find('p.my-profile strong')
    if reviewer_name is None:
        reviewer_name = "unknown"
    # 키워드 목록 가져오기
    keywords = browser.find_elements(By.CSS_SELECTOR, '.profile-keyword-list.on .list-item')
    text_list = [k.text.strip() for k in keywords]

    # reviewer_type 하나로 통합
    reviewer_type = ', '.join(text_list) if text_list else None

    # 탑리뷰어 분야 정보 추출
    try:
        top_reviewer_field = browser.find_element(By.CSS_SELECTOR, ".top-reviewer-text.on span").text.strip()
    except:
        top_reviewer_field = None

    # 자기소개글
    try:
        element = browser.find_element(By.CSS_SELECTOR, "p.top-reviewer-info.on")
        raw_text = element.get_attribute("innerText")
        lines = [line.strip() for line in raw_text.split("\n") if line.strip()]
        reviewer_introduce = ', '.join(lines)
    except:
        reviewer_introduce = None

    profile_info = {
        "reviewer_name": reviewer_name,
        "reviewer_type": reviewer_type,
        "top_reviewer_field": top_reviewer_field,
        "reviewer_introduce": reviewer_introduce
    }

    # 리뷰 정보 파싱
    smart_scroll_to_load_reviews(max_scrolls=150, delay=1.5)
    review_boxes = browser.find_elements(By.CSS_SELECTOR, '.rw-box')
    print(f"{reviewer_name} 리뷰 수: {len(review_boxes)}")

    all_reviews = []
    cleaned_reviews = []

    # CSV 파일 설정
    safe_reviewer_name = reviewer_name.replace("/", "_").replace("\\", "_")  # 이외의 무너지는 무엇이든 포함 가능
    filename = f"{safe_reviewer_name}_reviews.csv"
    file_exists = os.path.exists(filename)

    for idx, box in enumerate(review_boxes, start=1):
        review_data = parse_review_box(box)
        all_reviews.append(review_data)

        combined = {**profile_info, **review_data}
        cleaned = clean_for_csv(combined)
        cleaned_reviews.append(cleaned)

        if idx % save_interval == 0 or idx == len(review_boxes):
            with open(filename, mode='a', encoding='utf-8-sig', newline='') as file:
                writer = csv.DictWriter(file, fieldnames=field_order)
                if not file_exists:
                    writer.writeheader()
                    file_exists = True
                writer.writerows(cleaned_reviews)
            print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {reviewer_name} 중간 저장 완료! 리뷰 수: {idx}")
            cleaned_reviews = []

print("모든 리뷰어 크롤링 및 CSV 저장 완료!")

[2025-03-25 03:12:08] 리뷰 크롤링 시작!
스크롤 반복 중단, (총 105회 스크롤)
다블리 리뷰 수: 1036
[2025-03-25 03:33:05] 다블리 중간 저장 완료! 리뷰 수: 200
[2025-03-25 03:49:40] 다블리 중간 저장 완료! 리뷰 수: 400
[2025-03-25 04:07:04] 다블리 중간 저장 완료! 리뷰 수: 600
[2025-03-25 04:25:10] 다블리 중간 저장 완료! 리뷰 수: 800
[2025-03-25 04:44:31] 다블리 중간 저장 완료! 리뷰 수: 1000
[2025-03-25 04:47:47] 다블리 중간 저장 완료! 리뷰 수: 1036
[2025-03-25 04:47:47] 리뷰 크롤링 시작!
스크롤 반복 중단, (총 45회 스크롤)
올챙이포뇨 리뷰 수: 437
[2025-03-25 05:09:12] 올챙이포뇨 중간 저장 완료! 리뷰 수: 200
[2025-03-25 05:29:42] 올챙이포뇨 중간 저장 완료! 리뷰 수: 400
[2025-03-25 05:33:45] 올챙이포뇨 중간 저장 완료! 리뷰 수: 437
모든 리뷰어 크롤링 및 CSV 저장 완료!
